# Lab 3-2 Part 2: CI/CD Pipeline & Auto-Retrain

## 📋 실습 개요

| 항목 | 내용 |
|------|------|
| **소요시간** | 45분 |
| **난이도** | ⭐⭐⭐⭐ |
| **목표** | GitHub Actions CI/CD 및 자동 재학습 트리거 구현 |
| **사전 조건** | Part 1 완료, GitHub 계정 |

## 🎯 학습 목표

1. GitHub Actions 워크플로우 이해
2. CI Pipeline (테스트, 빌드)
3. CD Pipeline (배포 자동화)
4. Drift 기반 자동 재학습 트리거

---

## Step 0: 환경 설정

In [ ]:
!pip install requests pyyaml -q
print("✅ 패키지 설치 완료")

In [ ]:
import os
import json
import yaml
import requests
from datetime import datetime

# ============================================================
# ⚠️ 본인의 사용자 번호로 변경하세요!
# ============================================================
USER_NUM = "YOUR_USER_NUM"

USER_ID = f"user{USER_NUM}"
NAMESPACE = f"kubeflow-{USER_ID}"

# Prometheus URL
PROMETHEUS_URL = "http://prometheus.monitoring.svc.cluster.local:9090"

# Drift 임계값
MAE_THRESHOLD = 0.45
R2_THRESHOLD = 0.75

print("="*60)
print("  Lab 3-2 Part 2: CI/CD Pipeline & Auto-Retrain")
print("="*60)
print(f"\n👤 사용자: {USER_ID}")
print(f"📁 네임스페이스: {NAMESPACE}")

---

## Step 1: GitHub Actions CI/CD 아키텍처 이해

### 1.1 전체 아키텍처

In [ ]:
print("""
┌─────────────────────────────────────────────────────────────────────┐
│                    MLOps CI/CD Pipeline                              │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│   [Developer]                                                        │
│       │                                                              │
│       ▼                                                              │
│   ┌────────┐     ┌────────────────────────────────────────────┐     │
│   │  Push  │────►│              GitHub Actions                │     │
│   │ to Git │     │                                            │     │
│   └────────┘     │  ┌──────┐   ┌──────┐   ┌──────┐           │     │
│                  │  │ Test │──►│Build │──►│Deploy│           │     │
│                  │  │ Lint │   │ ECR  │   │KServe│           │     │
│                  │  └──────┘   └──────┘   └──────┘           │     │
│                  └────────────────────────────────────────────┘     │
│                                    │                                 │
│                                    ▼                                 │
│   ┌────────────────────────────────────────────────────────────┐    │
│   │                    Production Environment                   │    │
│   │  ┌─────────┐    ┌───────────┐    ┌────────────┐           │    │
│   │  │ KServe  │───►│  Metrics  │───►│ Prometheus │           │    │
│   │  │ Model   │    │  Exporter │    │            │           │    │
│   │  └─────────┘    └───────────┘    └─────┬──────┘           │    │
│   │                                        │                   │    │
│   │                                        ▼                   │    │
│   │                                  ┌──────────┐              │    │
│   │                                  │  Alert   │              │    │
│   │                                  │ Manager  │              │    │
│   │                                  └────┬─────┘              │    │
│   └───────────────────────────────────────│────────────────────┘    │
│                                           │                          │
│                                           ▼                          │
│                                    ┌────────────┐                    │
│                                    │  Retrain   │                    │
│                                    │  Trigger   │                    │
│                                    └────────────┘                    │
│                                           │                          │
│                                           ▼                          │
│                                    GitHub Actions                    │
│                                    (Retrain Job)                     │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
""")

### 1.2 CI/CD 단계별 설명

In [ ]:
print("""
📌 CI/CD Pipeline 단계
════════════════════════════════════════════════════════════════

🔵 CI (Continuous Integration)
──────────────────────────────────────────────────────────────────
1. Code Checkout
   - GitHub에서 소스 코드 체크아웃

2. Environment Setup
   - Python 환경 설정
   - 패키지 의존성 설치

3. Code Quality
   - Lint 검사 (flake8, black)
   - Type 검사 (mypy)

4. Unit Tests
   - pytest 실행
   - 코드 커버리지 측정

5. Integration Tests
   - API 테스트
   - 모델 추론 테스트

🟢 CD (Continuous Deployment)
──────────────────────────────────────────────────────────────────
6. Docker Build
   - 모델 서빙 컨테이너 빌드
   - Multi-arch 빌드 (amd64, arm64)

7. Push to Registry
   - AWS ECR에 이미지 푸시
   - 버전 태깅 (git sha, latest)

8. Deploy to KServe
   - InferenceService 업데이트
   - Canary 배포 (선택)

9. Verify Deployment
   - Health check
   - Smoke test

🔴 Auto-Retrain (Drift 감지 시)
──────────────────────────────────────────────────────────────────
10. Drift Detection
    - Prometheus 메트릭 모니터링
    - 임계값 초과 시 Alert 발생

11. Trigger Retrain
    - GitHub Actions workflow_dispatch 호출
    - 새 데이터로 모델 재학습

12. Re-deploy
    - 새 모델 검증 후 배포
    - A/B 테스트 (선택)
""")

---

## Step 2: CI Pipeline 워크플로우 분석

### 2.1 CI 워크플로우 YAML 구조

In [ ]:
ci_workflow = """
# .github/workflows/ci-test.yaml
name: CI - Test & Build

on:
  push:
    branches: [main, develop]
  pull_request:
    branches: [main]

env:
  PYTHON_VERSION: '3.9'

jobs:
  test:
    name: Run Tests
    runs-on: ubuntu-latest
    
    steps:
      # 1. 코드 체크아웃
      - name: Checkout code
        uses: actions/checkout@v4
      
      # 2. Python 환경 설정
      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: ${{ env.PYTHON_VERSION }}
          cache: 'pip'
      
      # 3. 의존성 설치
      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install -r requirements.txt
          pip install pytest pytest-cov flake8
      
      # 4. Lint 검사
      - name: Lint with flake8
        run: |
          flake8 src/ --count --select=E9,F63,F7,F82 --show-source --statistics
          flake8 src/ --count --exit-zero --max-complexity=10 --max-line-length=127 --statistics
      
      # 5. 단위 테스트
      - name: Run tests with coverage
        run: |
          pytest tests/ -v --cov=src --cov-report=xml --cov-report=html
      
      # 6. 커버리지 업로드
      - name: Upload coverage
        uses: codecov/codecov-action@v4
        with:
          file: ./coverage.xml
          fail_ci_if_error: false
"""

print(ci_workflow)

### 2.2 CI 워크플로우 핵심 요소

In [ ]:
print("""
📌 CI 워크플로우 핵심 요소
════════════════════════════════════════════════════════════════

1️⃣ Trigger (on:)
   - push: main/develop 브랜치에 푸시 시 실행
   - pull_request: main 브랜치로 PR 시 실행

2️⃣ Environment Variables (env:)
   - 워크플로우 전역에서 사용할 변수 정의
   - Python 버전, AWS 리전 등

3️⃣ Jobs
   - test: 테스트 실행 Job
   - 각 Job은 독립적인 Runner에서 실행

4️⃣ Steps
   - 순차적으로 실행되는 단계
   - uses: 미리 정의된 Action 사용
   - run: 쉘 명령어 실행

5️⃣ 주요 Actions
   - actions/checkout@v4: 코드 체크아웃
   - actions/setup-python@v5: Python 환경
   - codecov/codecov-action@v4: 커버리지 리포트

""")

---

## Step 3: CD Pipeline 워크플로우 분석

### 3.1 CD 워크플로우 YAML 구조

In [ ]:
cd_workflow = """
# .github/workflows/cd-deploy.yaml
name: CD - Deploy to KServe

on:
  workflow_run:
    workflows: ["CI - Test & Build"]
    types: [completed]
    branches: [main]

env:
  AWS_REGION: ap-northeast-2
  ECR_REPOSITORY: mlops-model-serving
  KSERVE_NAMESPACE: kubeflow-user01  # 사용자 네임스페이스

jobs:
  deploy:
    name: Build & Deploy
    runs-on: ubuntu-latest
    if: ${{ github.event.workflow_run.conclusion == 'success' }}
    
    steps:
      # 1. 코드 체크아웃
      - name: Checkout
        uses: actions/checkout@v4
      
      # 2. AWS 자격증명 설정
      - name: Configure AWS credentials
        uses: aws-actions/configure-aws-credentials@v4
        with:
          aws-access-key-id: ${{ secrets.AWS_ACCESS_KEY_ID }}
          aws-secret-access-key: ${{ secrets.AWS_SECRET_ACCESS_KEY }}
          aws-region: ${{ env.AWS_REGION }}
      
      # 3. ECR 로그인
      - name: Login to Amazon ECR
        id: login-ecr
        uses: aws-actions/amazon-ecr-login@v2
      
      # 4. Docker 빌드 & 푸시
      - name: Build and push Docker image
        env:
          ECR_REGISTRY: ${{ steps.login-ecr.outputs.registry }}
          IMAGE_TAG: ${{ github.sha }}
        run: |
          docker build -t $ECR_REGISTRY/$ECR_REPOSITORY:$IMAGE_TAG .
          docker build -t $ECR_REGISTRY/$ECR_REPOSITORY:latest .
          docker push $ECR_REGISTRY/$ECR_REPOSITORY:$IMAGE_TAG
          docker push $ECR_REGISTRY/$ECR_REPOSITORY:latest
      
      # 5. kubeconfig 설정
      - name: Setup kubeconfig
        run: |
          aws eks update-kubeconfig --name mlops-cluster --region $AWS_REGION
      
      # 6. KServe 배포
      - name: Deploy to KServe
        env:
          ECR_REGISTRY: ${{ steps.login-ecr.outputs.registry }}
          IMAGE_TAG: ${{ github.sha }}
        run: |
          # InferenceService YAML 생성
          cat <<EOF | kubectl apply -f -
          apiVersion: serving.kserve.io/v1beta1
          kind: InferenceService
          metadata:
            name: california-housing
            namespace: $KSERVE_NAMESPACE
          spec:
            predictor:
              containers:
                - name: model
                  image: $ECR_REGISTRY/$ECR_REPOSITORY:$IMAGE_TAG
                  ports:
                    - containerPort: 8080
          EOF
      
      # 7. 배포 검증
      - name: Verify deployment
        run: |
          kubectl rollout status deployment/california-housing-predictor -n $KSERVE_NAMESPACE --timeout=300s
          echo "✅ Deployment successful!"
"""

print(cd_workflow)

### 3.2 CD 워크플로우 핵심 요소

In [ ]:
print("""
📌 CD 워크플로우 핵심 요소
════════════════════════════════════════════════════════════════

1️⃣ Trigger (on: workflow_run)
   - CI 워크플로우 완료 후 자동 실행
   - conclusion: 'success'인 경우만 실행

2️⃣ AWS Integration
   - aws-actions/configure-aws-credentials: IAM 인증
   - aws-actions/amazon-ecr-login: ECR 로그인

3️⃣ Docker Build
   - Git SHA로 이미지 태깅
   - latest 태그도 함께 푸시

4️⃣ Kubernetes Deployment
   - EKS 클러스터 kubeconfig 설정
   - kubectl apply로 InferenceService 배포

5️⃣ GitHub Secrets
   - AWS_ACCESS_KEY_ID
   - AWS_SECRET_ACCESS_KEY
   - 민감 정보는 반드시 Secrets 사용

""")

---

## Step 4: 자동 재학습 트리거 구현

### 4.1 Drift 감지 함수

In [ ]:
def query_prometheus(query):
    """Prometheus 쿼리 실행"""
    try:
        response = requests.get(
            f"{PROMETHEUS_URL}/api/v1/query",
            params={"query": query},
            timeout=10
        )
        if response.status_code == 200:
            data = response.json()
            if data["status"] == "success":
                return data["data"]["result"]
    except Exception as e:
        print(f"Query error: {e}")
    return []

def check_model_drift(user_id):
    """
    모델 Drift 감지
    
    Returns:
        dict: {
            'drift_detected': bool,
            'mae': float,
            'r2': float,
            'drift_score': float,
            'reason': str
        }
    """
    # 현재 메트릭 조회
    mae_result = query_prometheus(f'model_mae_score{{user_id="{user_id}"}}')
    r2_result = query_prometheus(f'model_r2_score{{user_id="{user_id}"}}')
    
    if not mae_result or not r2_result:
        return {
            'drift_detected': False,
            'mae': None,
            'r2': None,
            'drift_score': 0,
            'reason': 'No metrics available'
        }
    
    current_mae = float(mae_result[0]["value"][1])
    current_r2 = float(r2_result[0]["value"][1])
    
    # Drift 판정
    mae_drift = current_mae > MAE_THRESHOLD
    r2_drift = current_r2 < R2_THRESHOLD
    
    # Drift Score 계산 (0~1, 높을수록 심각)
    mae_score = max(0, (current_mae - MAE_THRESHOLD) / MAE_THRESHOLD) if mae_drift else 0
    r2_score = max(0, (R2_THRESHOLD - current_r2) / R2_THRESHOLD) if r2_drift else 0
    drift_score = (mae_score + r2_score) / 2
    
    # 이유 생성
    reasons = []
    if mae_drift:
        reasons.append(f"MAE({current_mae:.4f}) > threshold({MAE_THRESHOLD})")
    if r2_drift:
        reasons.append(f"R²({current_r2:.4f}) < threshold({R2_THRESHOLD})")
    
    return {
        'drift_detected': mae_drift or r2_drift,
        'mae': current_mae,
        'r2': current_r2,
        'drift_score': drift_score,
        'reason': '; '.join(reasons) if reasons else 'Model performance is normal'
    }

print("✅ Drift 감지 함수 정의 완료")

### 4.2 현재 Drift 상태 확인

In [ ]:
print("="*60)
print(f"  {USER_ID} Drift 상태 확인")
print("="*60)

drift_status = check_model_drift(USER_ID)

print(f"\n📊 현재 메트릭:")
mae_str = f"{drift_status['mae']:.4f}" if drift_status['mae'] is not None else 'N/A'
r2_str = f"{drift_status['r2']:.4f}" if drift_status['r2'] is not None else 'N/A'
print(f"   MAE: {mae_str}")
print(f"   R²:  {r2_str}")
print(f"\n📏 임계값:")
print(f"   MAE Threshold: {MAE_THRESHOLD}")
print(f"   R² Threshold:  {R2_THRESHOLD}")
print(f"\n🔍 Drift 분석:")
print(f"   Drift Detected: {'🚨 YES' if drift_status['drift_detected'] else '✅ NO'}")
print(f"   Drift Score: {drift_status['drift_score']:.4f}")
print(f"   Reason: {drift_status['reason']}")

### 4.3 재학습 트리거 워크플로우

In [ ]:
retrain_workflow = """
# .github/workflows/retrain-model.yaml
name: Retrain Model on Drift

on:
  # 수동 트리거 (API로 호출 가능)
  workflow_dispatch:
    inputs:
      drift_score:
        description: 'Drift score that triggered retrain'
        required: true
        default: '0.5'
      user_id:
        description: 'User ID'
        required: true
        default: 'user01'

env:
  MLFLOW_TRACKING_URI: http://mlflow-server-service.mlflow-system:5000
  S3_BUCKET: mlops-data-bucket

jobs:
  retrain:
    name: Retrain Model
    runs-on: ubuntu-latest
    
    steps:
      - name: Checkout
        uses: actions/checkout@v4
      
      - name: Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: '3.9'
      
      - name: Install dependencies
        run: |
          pip install -r requirements.txt
      
      - name: Download latest data from S3
        env:
          AWS_ACCESS_KEY_ID: ${{ secrets.AWS_ACCESS_KEY_ID }}
          AWS_SECRET_ACCESS_KEY: ${{ secrets.AWS_SECRET_ACCESS_KEY }}
        run: |
          aws s3 cp s3://$S3_BUCKET/data/latest/ ./data/ --recursive
      
      - name: Train new model
        env:
          DRIFT_SCORE: ${{ github.event.inputs.drift_score }}
          USER_ID: ${{ github.event.inputs.user_id }}
        run: |
          python scripts/train_model.py \
            --experiment-name "drift-retrain-$USER_ID" \
            --drift-score $DRIFT_SCORE
      
      - name: Validate new model
        run: |
          python scripts/validate_model.py \
            --threshold ${{ env.MAE_THRESHOLD }}
      
      - name: Trigger deployment
        if: success()
        uses: peter-evans/repository-dispatch@v3
        with:
          event-type: model-retrained
          client-payload: '{"user_id": "${{ github.event.inputs.user_id }}"}'
"""

print(retrain_workflow)

### 4.4 재학습 트리거 시뮬레이션

In [ ]:
def simulate_retrain_trigger(user_id, drift_status):
    """
    재학습 트리거 시뮬레이션
    
    실제 환경에서는 GitHub API를 호출합니다.
    """
    print("="*60)
    print("  재학습 트리거 시뮬레이션")
    print("="*60)
    
    if not drift_status['drift_detected']:
        print(f"\n✅ Drift가 감지되지 않았습니다.")
        print("   재학습이 필요하지 않습니다.")
        return
    
    print(f"\n🚨 Drift 감지됨! 재학습을 트리거합니다.")
    print(f"\n📋 트리거 정보:")
    print(f"   User ID: {user_id}")
    print(f"   Drift Score: {drift_status['drift_score']:.4f}")
    print(f"   Reason: {drift_status['reason']}")
    
    print(f"\n🔄 GitHub Actions workflow_dispatch 호출...")
    print(f"   Repository: your-org/mlops-repo")
    print(f"   Workflow: retrain-model.yaml")
    print(f"   Inputs:")
    print(f"     - drift_score: {drift_status['drift_score']:.4f}")
    print(f"     - user_id: {user_id}")
    
    # 실제 GitHub API 호출 예시 (주석 처리)
    """
    import requests
    
    GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
    REPO = 'your-org/mlops-repo'
    
    response = requests.post(
        f'https://api.github.com/repos/{REPO}/actions/workflows/retrain-model.yaml/dispatches',
        headers={
            'Authorization': f'token {GITHUB_TOKEN}',
            'Accept': 'application/vnd.github.v3+json'
        },
        json={
            'ref': 'main',
            'inputs': {
                'drift_score': str(drift_status['drift_score']),
                'user_id': user_id
            }
        }
    )
    
    if response.status_code == 204:
        print("✅ 재학습 트리거 성공!")
    else:
        print(f"❌ 트리거 실패: {response.text}")
    """
    
    print(f"\n✅ 시뮬레이션 완료!")
    print(f"\n📌 실제 구현 시:")
    print(f"   1. GitHub Personal Access Token 필요")
    print(f"   2. Repository secrets에 TOKEN 등록")
    print(f"   3. workflow_dispatch API 호출")

# 시뮬레이션 실행
simulate_retrain_trigger(USER_ID, drift_status)

---

## Step 5: End-to-End 통합 워크플로우

### 5.1 전체 자동화 흐름

In [ ]:
print("""
📌 End-to-End MLOps 자동화 흐름
════════════════════════════════════════════════════════════════

1️⃣ Model Serving (KServe)
   └─► 프로덕션 환경에서 예측 수행

2️⃣ Metrics Collection (Prometheus)
   └─► MAE, R², Latency, Error Rate 수집

3️⃣ Monitoring (Grafana)
   └─► 실시간 대시보드 시각화

4️⃣ Alert Rules
   └─► MAE > 0.45 또는 R² < 0.75 시 Alert 발생

5️⃣ Alert Manager
   └─► Alert 수신 및 라우팅

6️⃣ Webhook Trigger
   └─► GitHub Actions workflow_dispatch 호출

7️⃣ Retrain Pipeline
   └─► 새 데이터로 모델 재학습
   └─► MLflow에 실험 기록

8️⃣ Model Validation
   └─► 새 모델 성능 검증

9️⃣ Auto Deploy
   └─► 검증 통과 시 자동 배포

🔟 Verify
   └─► Health check 및 Smoke test
   └─► 성능 개선 확인

""")

### 5.2 통합 테스트 스크립트

In [ ]:
def run_integration_test(user_id):
    """
    End-to-End 통합 테스트
    """
    print("="*60)
    print(f"  End-to-End 통합 테스트 - {user_id}")
    print("="*60)
    
    tests = [
        ("Prometheus 연결", lambda: requests.get(f"{PROMETHEUS_URL}/-/healthy", timeout=5).status_code == 200),
        ("메트릭 조회 (MAE)", lambda: len(query_prometheus(f'model_mae_score{{user_id="{user_id}"}}')) > 0),
        ("메트릭 조회 (R²)", lambda: len(query_prometheus(f'model_r2_score{{user_id="{user_id}"}}')) > 0),
        ("Drift 감지 기능", lambda: 'drift_detected' in check_model_drift(user_id)),
    ]
    
    passed = 0
    failed = 0
    
    print("\n테스트 실행 중...\n")
    
    for test_name, test_func in tests:
        try:
            if test_func():
                print(f"  ✅ {test_name}")
                passed += 1
            else:
                print(f"  ❌ {test_name}")
                failed += 1
        except Exception as e:
            print(f"  ❌ {test_name} - Error: {e}")
            failed += 1
    
    print("\n" + "="*60)
    print(f"  테스트 결과: {passed}/{passed+failed} 통과")
    print("="*60)
    
    if failed == 0:
        print("\n🎉 모든 테스트 통과!")
    else:
        print(f"\n⚠️ {failed}개 테스트 실패")
    
    return passed, failed

# 통합 테스트 실행
run_integration_test(USER_ID)

---

## ✅ Part 2 완료 체크리스트

- [ ] CI/CD 아키텍처 이해
- [ ] CI 워크플로우 구조 분석
- [ ] CD 워크플로우 구조 분석
- [ ] Drift 감지 함수 이해
- [ ] 재학습 트리거 메커니즘 이해
- [ ] End-to-End 통합 테스트 통과

---

## 📚 Lab 3-2 완료!

### 배운 내용 요약

1. **Part 1: Monitoring**
   - Prometheus 메트릭 조회
   - Model Drift 감지
   - Alert Rule 설정
   - Grafana 대시보드 활용

2. **Part 2: CI/CD**
   - GitHub Actions 워크플로우
   - CI Pipeline (테스트, 빌드)
   - CD Pipeline (배포 자동화)
   - 자동 재학습 트리거

### 다음 단계

- **Lab 3-3**: Model Optimization (ONNX, 양자화)
- **프로젝트 실습**: 팀별 End-to-End MLOps 파이프라인

---

© 2025 현대오토에버 MLOps Training